In [ ]:
import numpy as np
import pygad

### Variables globales del problema

In [ ]:
# Para VERTEX COVER

import networkx as nx
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\ENZYMES_g102.edges", nodetype=int)
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\aves-sparrowlyon-flock-season2-unw.edges", nodetype=int)
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\sparrow_unweighted.edges", nodetype=int)
G = nx.read_edgelist(r"..\vertex_cover_benchmarks\cats.edges", nodetype=int)
n = G.order()

In [ ]:
# Para KNAPSACK

values = []
weights = []

with open(r"..\knapsack_benchmarks\s004.kp") as f:
    n, W = [int(x) for x in next(f).split()] # read first line
    for line in f: # read rest of lines
        nums = line.split()
        values.append(int(nums[0]))
        weights.append(int(nums[1]))

### Funciones de fitness

In [ ]:
import random

def is_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            return False
    return True

def naive_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            node = random.choice([u, v])
            sol[node - 1] = 1
    return

def fitness_vertex_cover(sol):
    n = G.order()
    if not is_cover(sol, G):
        naive_cover(sol, G)
    return n - sum(sol)

def fitness_knapsack(sol):
    n = len(sol)
    fit = 0
    wei = 0
    for i in range(n):
        fit += values[i] * sol[i]
        wei += weights[i] * sol[i]
    while wei > W:
        sacar = random.randrange(0, n)
        wei -= weights[sacar] * sol[sacar]
        fit -= values[sacar] * sol[sacar]
        sol[sacar] = 0
    return fit

def fitness_func_vertex_cover(ga_instance, solution, solution_idx):
    return fitness_vertex_cover(solution)

def fitness_func_knapsack(ga_instance, solution, solution_idx):
    return fitness_knapsack(solution)

In [ ]:
# Hiperparámetros de GA

num_generations = 1000
num_parents_mating = 100

gene_space = [0, 1]

# Para VERTEX COVER:
#fitness_function = fitness_func_vertex_cover
# Para KNAPSACK:
fitness_function = fitness_func_knapsack

sol_per_pop = 100
num_genes = n

init_range_low = 0
init_range_high = 1

parent_selection_type = "tournament"
keep_parents = 0
keep_elitism = 5

crossover_type = "uniform"
crossover_probability = 0.7

mutation_type = "random"
mutation_probability = 0.1

gene_type = int

In [ ]:
# Mostrar los mejores resultados tras cada iteración
def on_gen(ga_instance):
    print("Generation: ", ga_instance.generations_completed)
    print("Fitness of the best solution:", ga_instance.best_solution()[1])

# Función que crea una instancia de objeto pygad.GA
def create_instance():
    return pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       crossover_probability=crossover_probability,
                       mutation_type=mutation_type,
                       mutation_probability=mutation_probability,
                       on_generation=on_gen,
                       gene_type=gene_type,
                       keep_elitism=keep_elitism,
                       gene_space=gene_space)

### N ejecuciones para VERTEX COVER O KNAPSACK

In [ ]:
# Elegir la ruta donde guardar los resultados:
output = open(r"..\results\kn4\GA.csv", "a")
# Elegir el número de ejecuciones:
num_ejec = 20
for _ in range(num_ejec):
    ga_instance = create_instance()
    ga_instance.run()
    solution, solution_fitness, _ = ga_instance.best_solution()
    output.write(f'{solution},')
    output.write(f'{solution_fitness}\n')
output.close()